In [1]:
! pip install kneed

In [2]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np
from math import *
from shapely.geometry import Point, Polygon
import geopandas as gpd
import sys
print(sys.path)
import math
from tqdm import tqdm
from pandas import DataFrame
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from kneed import KneeLocator
from sklearn.neighbors import NearestNeighbors, KDTree
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from threadpoolctl import threadpool_limits
from sklearn.preprocessing import StandardScaler

import pymongo
import json
import pandas as pd
from datetime import datetime, timedelta
from pandas import DataFrame
import matplotlib.pyplot as plt
from datetime import time
from tqdm import tqdm

from math import *
from shapely.geometry import Point, Polygon
import geopandas as gpd
import sys
print(sys.path)
import math
from pandas import DataFrame
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from kneed import KneeLocator
from sklearn.neighbors import KDTree
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

['/kaggle/lib/kagglegym', '/kaggle/lib', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/root/.local/lib/python3.10/site-packages', '/opt/conda/lib/python3.10/site-packages', '/root/src/BigQuery_Helper']
['/kaggle/lib/kagglegym', '/kaggle/lib', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/root/.local/lib/python3.10/site-packages', '/opt/conda/lib/python3.10/site-packages', '/root/src/BigQuery_Helper']


In [3]:
student_uids = ['u00', 'u01', 'u02', 'u03', 'u04', 'u05', 'u07', 'u08', 'u09', 'u10', 'u12', 'u13', 'u14', 'u15', 'u16',
                'u17', 'u18', 'u19', 'u20', 'u22', 'u23', 'u24', 'u25', 'u27', 'u30', 'u31', 'u32', 'u33', 'u34', 'u35',
                'u36', 'u39', 'u41', 'u42', 'u43', 'u44', 'u45', 'u46', 'u47', 'u49', 'u50', 'u51', 'u52', 'u53', 'u54',
                'u56', 'u57', 'u58', 'u59']
print(len(student_uids))

49


In [4]:
df_gps = pd.read_csv('/kaggle/input/studentlife-location-four-intervals/StudentLife_location_data.csv')
df_gps = df_gps.sort_values(by=['uid', 'timestamp'])
df_gps['timestamp'] = pd.to_datetime(df_gps['timestamp'])
del df_gps['Unnamed: 0']
df_gps.head(10)

,uid,timestamp,latitude,longitude,date,time_interval
156575,u00,2013-03-27 04:02:43,43.759135,-72.329240,2013-03-27,night
156576,u00,2013-03-27 04:22:24,43.759503,-72.329018,2013-03-27,night
156577,u00,2013-03-27 04:42:44,43.759134,-72.329238,2013-03-27,night
156578,u00,2013-03-27 05:02:43,43.759135,-72.329240,2013-03-27,night
156579,u00,2013-03-27 05:22:43,43.759135,-72.329240,2013-03-27,night
156580,u00,2013-03-27 05:42:43,43.759135,-72.329239,2013-03-27,night
156581,u00,2013-03-27 06:02:43,43.759134,-72.329241,2013-03-27,morning
156582,u00,2013-03-27 06:22:44,43.608052,-72.162171,2013-03-27,morning
156583,u00,2013-03-27 06:22:48,43.608052,-72.162171,2013-03-27,morning
156584,u00,2013-03-27 07:02:43,43.696293,-72.283302,2013-03-27,morning


In [5]:
# time spent in first frequently visited location for all student
# time spent in second frequently visited location for all students
# time spent in third frequently visited location for all students
# time spent in fourth frequently visited location for all students
# time spent in fifth frequently visited location for all students
# time spent in sixth frequently visited location for all students
# time spent in seventh frequently visited location for all students

In [6]:
def apply_kmeans_with_7_clusters(df, n_clusters=7):
    coords = df[['latitude', 'longitude']].values
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(coords)
    df['cluster'] = kmeans.labels_ + 1  
    return df

def ordinal(n):
    return "%d%s" % (n,"tsnrhtdd"[(n//10%10!=1)*(n%10<4)*n%10::4])

In [7]:
df_ = apply_kmeans_with_7_clusters(df_gps)
print(df_)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


        uid           timestamp   latitude  longitude        date  \
156575  u00 2013-03-27 04:02:43  43.759135 -72.329240  2013-03-27   
156576  u00 2013-03-27 04:22:24  43.759503 -72.329018  2013-03-27   
156577  u00 2013-03-27 04:42:44  43.759134 -72.329238  2013-03-27   
156578  u00 2013-03-27 05:02:43  43.759135 -72.329240  2013-03-27   
156579  u00 2013-03-27 05:22:43  43.759135 -72.329240  2013-03-27   
...     ...                 ...        ...        ...         ...   
198955  u59 2013-06-01 03:17:08  40.914261 -73.792014  2013-06-01   
198956  u59 2013-06-01 03:27:07  40.914236 -73.792047  2013-06-01   
198957  u59 2013-06-01 03:37:07  40.914250 -73.792028  2013-06-01   
198958  u59 2013-06-01 03:47:07  40.914247 -73.792007  2013-06-01   
198959  u59 2013-06-01 03:57:08  40.914270 -73.791962  2013-06-01   

       time_interval  cluster  
156575         night        1  
156576         night        1  
156577         night        1  
156578         night        1  
156579     

In [8]:
df_['cluster'].value_counts()

cluster
1    199872
7      1214
6       865
4       316
2       248
5       207
3       155
Name: count, dtype: int64

In [9]:
def calculate_time_spent(location_subset):
    time_spent = 0
    location_subset = location_subset.sort_values(by='timestamp')
    for i in range(len(location_subset) - 1):
        time_spent += (location_subset['timestamp'].iloc[i + 1] - 
                       location_subset['timestamp'].iloc[i]).total_seconds()
    return time_spent / 3600  

def time_spent_places_visited(df, uid):
    results = []
    user_df = df[df['uid'] == uid]
    unique_dates = user_df['date'].unique()

    for date in unique_dates:
        places_count = {
            'time_spent_morning_1stplace': 0,
            'time_spent_afternoon_1stplace': 0,
            'time_spent_evening_1stplace': 0,
            'time_spent_night_1stplace': 0,
            'time_spent_morning_2ndplace': 0,
            'time_spent_afternoon_2ndplace': 0,
            'time_spent_evening_2ndplace': 0,
            'time_spent_night_2ndplace': 0,
            'time_spent_morning_3rdplace': 0,
            'time_spent_afternoon_3rdplace': 0,
            'time_spent_evening_3rdplace': 0,
            'time_spent_night_3rdplace': 0,
            'time_spent_morning_4thplace': 0,
            'time_spent_afternoon_4thplace': 0,
            'time_spent_evening_4thplace': 0,
            'time_spent_night_4thplace': 0,
            'time_spent_morning_5thplace': 0,
            'time_spent_afternoon_5thplace': 0,
            'time_spent_evening_5thplace': 0,
            'time_spent_night_5thplace': 0,
            'time_spent_morning_6thplace': 0,
            'time_spent_afternoon_6thplace': 0,
            'time_spent_evening_6thplace': 0,
            'time_spent_night_6thplace': 0,
            'time_spent_morning_7thplace': 0,
            'time_spent_afternoon_7thplace': 0,
            'time_spent_evening_7thplace': 0,
            'time_spent_night_7thplace': 0
        }
        
        for time_interval in ['morning', 'afternoon', 'evening', 'night']:
            subset = user_df[(user_df['date'] == date) & 
                             (user_df['time_interval'] == time_interval)]
            
            for cluster_num in range(1, 8):
                location_subset = subset[subset['cluster'] == cluster_num]
                if not location_subset.empty:
                    places_count[f'time_spent_{time_interval}_{ordinal(cluster_num)}place'] = calculate_time_spent(location_subset)
        
        results.append({
            'uid': uid,
            'date': date,
            **places_count
        })
        
    results_df = pd.DataFrame(results)
    
    return results_df

In [10]:
result_3 = pd.DataFrame()

for uid in tqdm(student_uids):
    df_ = time_spent_places_visited(df_gps, uid)
    result_3 = pd.concat([result_3, df_], ignore_index=True)

result_3

100%|██████████| 49/49 [01:06<00:00,  1.35s/it]


,uid,date,time_spent_morning_1stplace,time_spent_afternoon_1stplace,time_spent_evening_1stplace,time_spent_night_1stplace,time_spent_morning_2ndplace,time_spent_afternoon_2ndplace,time_spent_evening_2ndplace,time_spent_night_2ndplace,...,time_spent_evening_5thplace,time_spent_night_5thplace,time_spent_morning_6thplace,time_spent_afternoon_6thplace,time_spent_evening_6thplace,time_spent_night_6thplace,time_spent_morning_7thplace,time_spent_afternoon_7thplace,time_spent_evening_7thplace,time_spent_night_7thplace
0,u00,2013-03-27,5.666667,5.666667,5.668333,1.666667,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
1,u00,2013-03-28,5.767778,5.667500,5.666667,5.668333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
2,u00,2013-03-29,5.665278,5.666667,5.666667,5.001389,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
3,u00,2013-03-30,0.666667,0.333333,5.666667,5.666667,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
4,u00,2013-03-31,5.666667,5.666667,5.666667,5.666667,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2819,u59,2013-05-28,5.833889,5.834444,5.967500,5.833333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
2820,u59,2013-05-29,5.549722,5.833056,5.848333,5.856111,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
2821,u59,2013-05-30,5.833333,5.833333,5.704167,5.893056,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
2822,u59,2013-05-31,5.892500,5.833611,2.038611,5.866667,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.166111,0.000000,0.0,0.0,1.213333,0.0


In [11]:
result_3.columns

Index(['uid', 'date', 'time_spent_morning_1stplace',
       'time_spent_afternoon_1stplace', 'time_spent_evening_1stplace',
       'time_spent_night_1stplace', 'time_spent_morning_2ndplace',
       'time_spent_afternoon_2ndplace', 'time_spent_evening_2ndplace',
       'time_spent_night_2ndplace', 'time_spent_morning_3rdplace',
       'time_spent_afternoon_3rdplace', 'time_spent_evening_3rdplace',
       'time_spent_night_3rdplace', 'time_spent_morning_4thplace',
       'time_spent_afternoon_4thplace', 'time_spent_evening_4thplace',
       'time_spent_night_4thplace', 'time_spent_morning_5thplace',
       'time_spent_afternoon_5thplace', 'time_spent_evening_5thplace',
       'time_spent_night_5thplace', 'time_spent_morning_6thplace',
       'time_spent_afternoon_6thplace', 'time_spent_evening_6thplace',
       'time_spent_night_6thplace', 'time_spent_morning_7thplace',
       'time_spent_afternoon_7thplace', 'time_spent_evening_7thplace',
       'time_spent_night_7thplace'],
      dtype

In [12]:
max_values = result_3.max()
print("\nMax values for each column:")
print(max_values)


Max values for each column:
uid                                     u59
date                             2013-06-01
time_spent_morning_1stplace        5.980833
time_spent_afternoon_1stplace      5.993333
time_spent_evening_1stplace        5.993333
time_spent_night_1stplace          5.993333
time_spent_morning_2ndplace        5.666667
time_spent_afternoon_2ndplace      5.666667
time_spent_evening_2ndplace        5.666667
time_spent_night_2ndplace          5.687778
time_spent_morning_3rdplace        4.683611
time_spent_afternoon_3rdplace      5.063333
time_spent_evening_3rdplace        5.566944
time_spent_night_3rdplace          5.666667
time_spent_morning_4thplace        5.875833
time_spent_afternoon_4thplace      5.833333
time_spent_evening_4thplace        5.833333
time_spent_night_4thplace          5.833333
time_spent_morning_5thplace        5.666667
time_spent_afternoon_5thplace      5.705278
time_spent_evening_5thplace        5.333333
time_spent_night_5thplace          5.751667
tim

In [13]:
result_3.to_csv('location_features_studentlife_second_half.csv')